## Lectura de los datos

#### Librerias

In [1]:
import pandas as pd
import numpy as np
import keras_metrics
import tensorflow as tf
import shutil, os, sys
import matplotlib.pyplot as plt


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers, layers
from tensorflow.keras import utils as k
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory


#### Datos

In [2]:

file= 'metadata.csv'
dt=pd.read_csv(file, sep=",")

dt.head(5)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,NaN,NaN,NaN,NaN,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


### Preprocesamiento

Vamos a ver en que proporcion encontramos las muestras de covid y las de otras enfermedades para que la red no se sobre entrene con un solo tipo de imagenes.

In [3]:
covid=[]
otro=[]

for i,r in dt.iterrows():
    if r['finding'] == 'COVID-19':
        covid.append(r)
    else:
        otro.append(r)

print("Co: ",len(covid) )
print("Otro: ",len(otro) )
print("Proporcion de Covid: ", len(covid)/(len(covid)+len(otro))*100)
    


Co:  296
Otro:  76
Proporcion de Covid:  79.56989247311827


Para que la funcion de keras: <i>image_dataset_from_directory</i> funcione para poder entrenar como queremos, separamos las fotos en dos capetas, una para covid y otra para el resto de enfermedades.

Como hemos visto hay muchas más radiografias de covid que de las otras enfermedades así que vamos a igualar el numero de muestras para cada clase.

##### Creamos los directorios

In [4]:
carpeta = os.getcwd()+os.sep+'images'

try:
    os.stat(carpeta+os.sep+'covid')
except:
    os.mkdir(carpeta+os.sep+'covid')
    
try:
    os.stat(carpeta+os.sep+'otro')
except:
    os.mkdir(carpeta+os.sep+'otro')

if not os.path.isdir(os.getcwd()+os.sep+'perdidos') :
    os.mkdir(os.getcwd()+os.sep+'perdidos')
    if not os.path.isdir(os.getcwd()+os.sep+'perdidos'+os.sep+'covid'):
        os.mkdir(os.getcwd()+os.sep+'perdidos'+os.sep+'covid')
        if not os.path.isdir(os.getcwd()+os.sep+'perdidos'+os.sep+'otro'):
            os.mkdir(os.getcwd()+os.sep+'perdidos'+os.sep+'otro')
            

In [5]:
dimensiones=[960,540]

dirs = os.listdir( carpeta )

def resize():
    for item in dirs:
        if os.path.isfile(carpeta+item):
            im = Image.open(carpeta+item)
            f, e = os.path.splitext(carpeta+item)
            imResize = im.resize((dimensiones[0],dimensiones[1]), Image.ANTIALIAS)
            imResize.save(f + '.jpg', 'JPEG', quality=90)
            
resize()

##### Movemos las imagenes

In [6]:

foto=''
destino=''
ruta=carpeta+destino
noEstanco=[]
noEstanot=[]

numcovid=0  #flag para equilibrar las muestras
numvalco=0
numvalotro=0
flag=0

for i,r in dt.iterrows():
    flag=0
    foto=r['filename']
    if r['finding'] == 'COVID-19':
        if numcovid<(len(otro)+10):
            destino='covid'
            numcovid+=1
        elif numvalco < 6:
            if os.path.exists(ruta+os.sep+foto):
                shutil.move(ruta+os.sep+foto,os.getcwd()+os.sep+'perdidos'+os.sep+'covid')
                numvalco+=1
                flag=1
                noEstanco.append(foto)
    else:
        if numvalotro < 6:
            if os.path.exists(ruta+os.sep+foto):
                shutil.move(ruta+os.sep+foto,os.getcwd()+os.sep+'perdidos'+os.sep+'otro')
                numvalotro+=1
                flag=1
                noEstanot.append(foto)
        destino='otro'
    if os.path.exists(ruta+os.sep+foto) and flag!=1:
        shutil.move(ruta+os.sep+foto,ruta+os.sep+destino)
    
    #print(ruta+os.sep+foto)
    

##### Guardamos las imagenes en memoria

In [7]:


img_fit= image_dataset_from_directory(directory="images", image_size=(dimensiones[0],dimensiones[1]), subset="training", validation_split=0.2, seed=1234,
                                      labels='inferred')
clases=img_fit.class_names
img_val=image_dataset_from_directory(directory="images", image_size=(dimensiones[0],dimensiones[1]),subset="validation",validation_split=0.2, seed=1234,
                                      labels='inferred')
img=image_dataset_from_directory(directory="perdidos", image_size=(dimensiones[0],dimensiones[1]),
                                      labels='inferred')

#PARA MEJORAR EL RENDIMIENTO
AUTOTUNE = tf.data.AUTOTUNE
img_fit=img_fit.cache().prefetch(buffer_size=AUTOTUNE)
img_val=img_val.cache().prefetch(buffer_size=AUTOTUNE)

Normalizar= layers.experimental.preprocessing.Rescaling(1./255, input_shape=(dimensiones[0],dimensiones[1], 3))


Found 154 files belonging to 2 classes.
Using 124 files for training.
Found 154 files belonging to 2 classes.
Using 30 files for validation.
Found 11 files belonging to 2 classes.


##### Aumentamos la caantidad de muestras que tenemos

In [8]:

aumento = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(dimensiones[0],dimensiones[1],3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.15),
  ]
)

pd= layers.ZeroPadding2D(padding=(1, 1))

### CREACION DEL MODELO

In [9]:
model= Sequential([
    aumento,
    Normalizar,
    pd,
    layers.Conv2D(filters=16,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=5),
    layers.Conv2D(filters=32,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=5),
    layers.Conv2D(filters=64,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=5),
    layers.Flatten(),
    Dense(256,'relu'),
    Dense(1,'sigmoid')

])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 960, 540, 3)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 960, 540, 3)       0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 962, 542, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 960, 540, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 192, 108, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 190, 106, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 21, 32)       

### ENTRENAMIENTO

In [ ]:
opti= optimizers.Adam(learning_rate=0.35)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
historico=model.fit(img_fit, validation_data= img_val, epochs= 5)

Epoch 1/5
4/4 [==============================] - 25s 5s/step - loss: 0.0000e+00 - accuracy: 0.5434 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 2/5
4/4 [==============================] - 18s 5s/step - loss: 0.0000e+00 - accuracy: 0.4994 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 3/5
4/4 [==============================] - 18s 5s/step - loss: 0.0000e+00 - accuracy: 0.4994 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 4/5
4/4 [==============================] - 18s 5s/step - loss: 0.0000e+00 - accuracy: 0.4994 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 5/5
1/4 [======>.......................] - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.4062

In [ ]:
#prediccion=model.predict(img)
#print(prediccion)
#for i in range(len(prediccion)):
#    score = tf.nn.softmax(prediccion[i])
#    print(
#        "La radiografia pertenece a la  {} con {:.2f} de precision."
#        .format(clases[np.argmax(score)], 100 * np.max(score))
#    )

In [ ]:
rutas=[ os.getcwd()+os.sep+'perdidos'+os.sep+'covid'+os.sep ,os.getcwd()+os.sep+'perdidos'+os.sep+'otro'+os.sep]
nombres=noEstanco+noEstanot
for i in range(len(rutas)):
    for j in range(len(nombres)):
        if os.path.exists(rutas[i]+nombres[j]): 
            image = tf.keras.preprocessing.image.load_img(rutas[i]+nombres[j], target_size=(dimensiones[0],dimensiones[1]))
            input_arr = keras.preprocessing.image.img_to_array(image)
            input_arr = np.array([input_arr]) 
            prediccion = model.predict(input_arr)
            print(prediccion)
            score = tf.nn.softmax(prediccion[0])
            print(
                "La radiografia pertenece a la  {} con {:.2f} de precision."
                .format(clases[np.argmax(score)], 100 * np.max(score))
            )


#sunflower_url = os.getcwd()+os.sep+'perdidos'+os.sep+'otro'+os.sep+'SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg'
#sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)
#
#img = keras.preprocessing.image.load_img(
#    sunflower_url, target_size=(dimensiones[0],dimensiones[1])
#)
#img_array = keras.preprocessing.image.img_to_array(img)
#img_array = tf.expand_dims(img_array, 0) # Create a batch
#
#predictions = model.predict(img_array)
#score = tf.nn.softmax(predictions[0])
#
#print(
#    "This image most likely belongs to {} with a {:.2f} percent confidence."
#    .format(class_names[np.argmax(score)], 100 * np.max(score))
#)

In [ ]:
prediccion